# AAI-520-02 Final Team Project: Multi-Agent System

#### Contributors:
- Alexander J Padin
- Thomas Geraci
- Ali Mohtat

## Imports

In [3]:
import os
from typing import Dict, Any, List
from dotenv import load_dotenv
import pandas as pd
import yfinance as yf
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool 
from typing import Dict, Any
from IPython.display import display, Markdown
load_dotenv() # read .env variable with OpenAI API Key

True

## Tools
Tools are modular, executable functions that agents can call to perform tasks (e.g., fetching data or interacting with APIs). In CrewAI, we wrap a function with `@tool("Tool Name")`, which registers it with metadata. The agent can then decide when to call which tool and how to use its output.  

### Tools in this project  
- `yf_prices(ticker, period='90d', interval='1d')`: fetch recent OHLCV (Open-High-Low-Close-Volume) price data  
- `yf_fundamentals(ticker)`: retrieve a compact fundamentals snapshot (e.g. market cap, P/E, EPS)  
- `yf_dividends(ticker, limit=5)`: fetch recent dividend events (up to N)  
- `yf_calendar(ticker)`: get upcoming or recent calendar events (e.g. earnings)  

### OHLCV Fetcher
This tool fetches recent OHLCV (Open, High, Low, Close, Volume) data for a given stock ticker over a specified time period (90 days) using `yfinance`. It returns up to the last 10 rows of data (the “tail”) plus metadata indicating how many rows were available. If no data is found, it returns an empty result with zero rows.

In [6]:
@tool("Fetch recent OHLCV prices from Yahoo Finance")
def yf_prices(ticker: str, period: str = "90d", interval: str = "1d"):
    """Returns recent OHLCV prices from Yahoo Finance""" # need docstring for tools
    
    tk = yf.Ticker(ticker)
    df = tk.history(period=period, interval=interval, auto_adjust=False)
    
    if df is None or df.empty: # if empty
        return {"ticker": ticker, "period": period, "interval": interval, "rows": 0, "tail10": []}
        
    out = df.reset_index().tail(10).to_dict(orient="records")
    return {"ticker": ticker, "period": period, "interval": interval, "rows": len(df), "tail10": out}

### Fundamentals Snapshot Fetcher
This tool fetches a compact fundamentals snapshot for a given ticker using `yfinance`. It tries to use `fast_info` (a lightweight subset of metadata) when available; if not, it falls back to `Ticker.info`. The returned dict includes fields like last price, market cap, P/E, EPS, 52-week high/low, and others if available.

In [8]:
@tool("Fetch a fundamentals snapshot from Yahoo Finance")
def yf_fundamentals(ticker: str) -> Dict[str, Any]:
    """Returns a fundamentals snapshot from Yahoo Finance"""
    
    tk = yf.Ticker(ticker)
    info = getattr(tk, "fast_info", None)
    
    if info:
        # if fast_info exists, use selected attributes
        attrs = [
            "last_price", "market_cap", "currency", 
            "year_high", "year_low", "shares_outstanding"
        ]
        fi = {k: getattr(info, k, None) for k in attrs}
        
    else: # else, get data form Ticker.info
        try:
            raw = tk.info
        except Exception:
            raw = {}
        fi = {
            "last_price": raw.get("currentPrice"),
            "market_cap": raw.get("marketCap"),
            "currency": raw.get("currency"),
            "year_high": raw.get("fiftyTwoWeekHigh"),
            "year_low": raw.get("fiftyTwoWeekLow"),
            "shares_outstanding": raw.get("sharesOutstanding"),
            "trailing_pe": raw.get("trailingPE"),
            "forward_pe": raw.get("forwardPE"),
            "eps": raw.get("trailingEps") or raw.get("epsTrailingTwelveMonths"),
            "short_name": raw.get("shortName"),
            "beta": raw.get("beta"),
        }
        
    return {"ticker": ticker, "fundamentals": fi}

### Dividends History Fetcher
This tool retrieves the most recent dividend events for a given ticker via `yfinance`. It returns up to a given number of past dividends (5), each with a date and dividend amount. If the ticker has no dividends or the data is empty, it returns an empty list.

In [10]:
@tool("Fetch dividends history (compact)")
def yf_dividends(ticker: str, limit: int = 5) -> Dict[str, Any]:
    """Returns dividends history (compact)"""
    
    tk = yf.Ticker(ticker)
    s = tk.dividends
    
    if s is None or s.empty: # if no dividends 
        return {"ticker": ticker, "dividends": []}
        
    df = s.reset_index().rename(columns={"Date": "date", "Dividends": "dividend"})
    return {"ticker": ticker, "dividends": df.tail(limit).to_dict(orient="records")}

### Earnings Calendar & Next Events Fetcher
This tool attempts to fetch upcoming or recent calendar events (e.g. earnings, dividends) for a ticker via `yfinance.Ticker.calendar`. It returns a structured list of event names and values if available; otherwise it returns an empty list.

In [12]:
@tool("Fetch earnings calendar / next events")
def yf_calendar(ticker: str) -> Dict[str, Any]:
    """Returns earnings calendar / next events"""
    
    tk = yf.Ticker(ticker)
    
    try:
        cal = tk.calendar
        if cal is None or cal.empty: # if no events
            return {"ticker": ticker, "calendar": []}
            
        df = cal.reset_index().rename(columns={"index": "event", 0: "value"})
        return {"ticker": ticker, "calendar": df.to_dict(orient="records")}
    except Exception: # if no success with yfinance endpoint
        return {"ticker": ticker, "calendar": []}

## Agents
An agent is an autonomous AI entity defined by a role, goal, a backstory (context) and tools. It reasons, plans, and acts on tasks using its tools, and can collaborate within a crew.  

### Agents in the Crew:
- Planner: Designs a structured, step-by-step plan (5–7 bullets) for how to research a given ticker using the Yahoo Finance tools.
- Researcher: Executes the plan by calling the Yahoo Finance tools (prices, fundamentals, dividends, calendar) and synthesizes findings, including a Buy/Sell/Hold recommendation and compact data summary.
- Reviewer: Reviews the Researcher's draft output and generates feedback and improvement suggestions (a review plan) for the next run.
- Optimizer: Takes the Researcher's work plus Reviewer feedback; modifies and refines the draft accordingly, and stores lessons learned to improve future runs.

### Planner Agent
This agent's role is to design a compact, tactical research plan for a given ticker. It does not fetch data itself, instead it creates a 5-7 step sequence that a Researcher agent can follow using only the Yahoo Finance tools (prices, fundamentals, dividends, calendar).  

In [15]:
planner = Agent(
    role="Planner", # role
    goal=( # goal prompt
        "Design a concise, step-by-step plan (5–7 bullets) for researching {ticker} "
        "using only the Yahoo Finance tools available to the Researcher."
    ),
    backstory=( # context
        "You are a methodical planning specialist. Your job is to outline an efficient "
        "sequence of steps that the Researcher can follow to gather price context, "
        "key fundamentals, dividends, upcoming events, and any red-flag checks."
    ),
    tools=[], # no tools used
    allow_delegation=False, # no delegation to other agents
    memory=False, # no retain of memory (maybe?)
    verbose=True, # enable logging of internal reasoning
    llm="gpt-4o", # the LLM to be used; gpt-4o-mini is too small for this task
    max_iter=2, # number of reasoning iterations
    max_rpm=30, # max request per mins
)

### Researcher Agent
This agent uses Yahoo Finance tools (prices, fundamentals, dividends, calendar) to generate a compact, data-driven analysis of a given stock. Its output includes a ~200-word synthesis, a Buy/Sell/Hold recommendation, and a concise data snapshot.  

In [17]:
researcher = Agent(
    role="Researcher",
    goal=(
        "Use the provided Yahoo Finance tools to gather a concise, data-backed snapshot "
        "for a given stock ticker. Produce a ~200-word synthesis, a Buy/Sell/Hold recommendation, "
        "and a compact data summary."
    ),
    backstory=(
        "You are a pragmatic equity researcher. You rely only on Yahoo Finance via the tools provided "
        "and call out any missing or uncertain data. Keep outputs crisp and actionable."
    ),
    tools=[yf_prices, yf_fundamentals, yf_dividends, yf_calendar],
    allow_delegation=False,
    memory=False,
    verbose=True,
    llm="gpt-4o",
    max_iter=3,
    max_rpm=30,
)

## Tasks
A Task is an assignment that an Agent must complete. It contains everything needed to perform such as the description telling the agent what to do, the agent responsible for executing it, expected outputs/format, context and tools that the agent can use during task execution. 

### Planning Task 
This  Task directs the Planner Agent to produce a structured 5–7 step **RESEARCH_PLAN** for the Researcher agent, using only the allowed Yahoo Finance tools (`yf_prices`, `yf_fundamentals`, `yf_dividends`, `yf_calendar`).  

In [20]:
PLANNING_PROMPT = """You are the Equity Research Planner.

Ticker: {ticker}

INSTRUCTIONS:
- Draft a 5–7 step RESEARCH_PLAN for the Researcher that uses ONLY these tools:
  - yf_prices(ticker, period='90d', interval='1d')
  - yf_fundamentals(ticker)
  - yf_dividends(ticker, limit=5)
  - yf_calendar(ticker)
- Order steps logically (price context → fundamentals → dividends → calendar → red flags).
- Be specific about periods/parameters the Researcher should try.
- If data could be missing, note a fallback (e.g., try a longer period).

OUTPUT:
**RESEARCH_PLAN** — numbered bullets
"""

# creates a CrewAI task
plan_research = Task(
    description=PLANNING_PROMPT, # prompt with the instructions for the task
    agent=planner, # the agent responsable for this task (Planner)
    expected_output="RESEARCH_PLAN" # the expected output
)

### Research Task: 
This Task directs the Researcher to produce a data-driven SYNTHESIS, a Buy/Sell/Hold recommendation with rationale, and a compact DATA SNAPSHOT. It instructs the agent to consume a prior RESEARCH_PLAN or draft one itself, then use the Yahoo Finance tools (`yf_prices`, `yf_fundamentals`, `yf_dividends`, `yf_calendar`) to pull data and structure the output accordingly.

In [22]:
RESEARCH_PROMPT = """You are the Equity Researcher.

Ticker: {ticker}

TOOLS AVAILABLE:
- yf_prices(ticker, period='90d', interval='1d'): recent OHLCV
- yf_fundamentals(ticker): compact fundamentals snapshot
- yf_dividends(ticker, limit=5): recent dividends
- yf_calendar(ticker): next/recent calendar entries (e.g., earnings)

INSTRUCTIONS:
0) Follow the RESEARCH_PLAN provided in context. If no plan is present, quickly draft a brief one before proceeding.
1) Call the tools you need to understand {ticker}'s recent behavior and snapshot.
2) Summarize price context, key fundamentals (best-effort), any dividends, and upcoming events if present.
3) If a tool returns empty/missing fields, acknowledge it and suggest a reasonable fallback (e.g., longer period).
4) Produce a Buy/Sell/Hold recommendation.

OUTPUT:
**SYNTHESIS (~150–250 words)**
**RECOMMENDATION** — a one-word classification (BUY/SELL/HOLD) plus one short sentence of rationale, and include "recommendation" inside the JSON snapshot.
**DATA SNAPSHOT** — compact JSON
"""

research_ticker = Task(
    description=RESEARCH_PROMPT,
    agent=researcher, # executed by Researcher Agent
    expected_output="SYNTHESIS, RECOMMENDATION, DATA SNAPSHOT",
    context=[plan_research] # passes the Planner's output to the Researcher as Context
)

## Crew Setup
Here we create a Crew that coordinates the agents and the tasks.

When `crew.kickoff()` is called, the Planner will produce a plan, then the Researcher uses that plan to drive its analysis....  


In [24]:
crew = Crew(
    agents=[planner, researcher], # list agents        
    tasks=[plan_research, research_ticker], #  list of tasks to execute in order
    process=Process.sequential, # task to run one after another
    verbose=True # enable logging
)

crew

Crew(id=e011ed7b-2623-4f1b-909f-1d1b36c71c6f, process=Process.sequential, number_of_agents=2, number_of_tasks=2)

In [25]:
# execute the crew's tasks in sequence using an input
result = crew.kickoff(inputs={"ticker": "AAPL"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e011ed7b-2623-4f1b-909f-1d1b36c71c6f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Task: You are the Equity Research Planner.                                                                     │
│                                                                                                                 │
│  Ticker: AAPL                                                                                                   │
│                                                                                                                 │
│  INSTRUCTIONS:                                                                                                  │
│  - Draft a 5–7 step RESEARCH_PLAN for the Researcher that uses ONLY these tools:                                │
│    - yf_prices(ticker, period='90d', interval='1d')                                                             │
│    - yf_fundamentals(ticker)                                                                                    │
│    - yf_dividends(ticker, limit=5)                                                                              │
│    - yf_calendar(ticker)                                                                                        │
│  - Order steps logically (price context → fundamentals → dividends → calendar → red flags).                     │
│  - Be specific about periods/parameters the Researcher should try.                                              │
│  - If data could be missing, note a fallback (e.g., try a longer period).                                       │
│                                                                                                                 │
│  OUTPUT:                                                                                                        │
│  **RESEARCH_PLAN** — numbered bullets                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **RESEARCH_PLAN** — numbered bullets                                                                           │
│                                                                                                                 │
│  1. **Price Context:**                                                                                          │
│     - Use `yf_prices('AAPL', period='90d', interval='1d')` to gather daily price data for the last 90 days.     │
│     - Examine the trend for any noticeable fluctuations or patterns in the stock price that may inform your     │
│  analysis.                                                                                                      │
│                                                                                                                 │
│  2. **Key Fundamentals:**                                                                                       │
│     - Access `yf_fundamentals('AAPL')` to review key company fundamentals including metrics such as PE ratio,   │
│  market cap, and revenue.                                                                                       │
│     - Identify any fundamental strengths or weaknesses compared to industry peers.                              │
│                                                                                                                 │
│  3. **Dividends:**                                                                                              │
│     - Retrieve recent dividend data using `yf_dividends('AAPL', limit=5)`.                                      │
│     - Analyze these dividends to understand Apple's dividend yield, consistency, and any recent changes in      │
│  payout.                                                                                                        │
│                                                                                                                 │
│  4. **Upcoming Events:**                                                                                        │
│     - Utilize `yf_calendar('AAPL')` to obtain information on upcoming events such as earnings announcements or  │
│  shareholder meetings.                                                                                          │
│     - Identify any events that may impact the stock price.                                                      │
│                                                                                                                 │
│  5. **Red Flag Checks:**                                                                                        │
│     - Re-assess all gathered data for any red flags, such as sudden changes in fundamentals or dividends, or    │
│  unusual price volatility.                                                                                      │
│     - If necessary data is missing, consider extending the period or reviewing industry reports for additional  │
│  insights.                                                                                                      │
│                                                                                                                 │
│  By following these steps, the Researcher will comprehensively gather necessary data on AAPL using Yahoo        │
│  Finance tools, allowing them to build a well-informed 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9cfee04c-75db-4438-b38a-a439bdca18ec                                                                     │
│  Agent: Planner                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Maximum iterations reached. Requesting final answer.


Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **SYNTHESIS** — Apple's stock has shown an upward trend over the past few days, recently reaching a high of    │
│  $258.79 before a slight decline. It maintains a significant market cap of approximately $3.8 trillion,         │
│  consistent with its leading position in the tech industry. Although the exact P/E ratio is not available, the  │
│  company's valuation relative to its historical performance can be check against industry norms or analyst      │
│  reports. Dividend payouts have been stable, showing moderate increments, reflecting a reliable shareholder     │
│  return strategy. No forthcoming corporate events on the horizon could act as immediate catalysts for stock     │
│  price movement, reducing near-term volatility risks.                                                           │
│                                                                                                                 │
│  **RECOMMENDATION** — HOLD: Given the stability in dividends, the resilience of its market cap, and the         │
│  absence of immediate market-moving events, it's prudent to adopt a hold strategy awaiting further fundamental  │
│  data to trigger a re-evaluation.                                                                               │
│                                                                                                                 │
│  **DATA SNAPSHOT** —                                                                                            │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "price_trend": "upward",                                                                                     │
│    "recent_price": 255.45,                                                                                      │
│    "market_cap": 3803334770730.35,                                                                              │
│    "dividends": [                                                                                               │
│      {"date": "2025-08-11", "dividend": 0.26},                                                                  │
│      {"date": "2025-05-12", "dividend": 0.26},                                                                  │
│      {"date": "2025-02-10", "dividend": 0.25},                                                                  │
│      {"date": "2024-11-08", "dividend": 0.25},                                                                  │
│      {"date": "2024-08-12", "dividend": 0.25}                                                                   │
│    ],                                                                                                           │
│    "upcoming_events": [],                                                                                       │
│    "recommendation": "HOLD"                                                                                     │
│  }                                                                                                              │
│  ```                                                                                                            │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a7a7c0c2-c62b-44fd-8441-a8fc0ef6ffbb                                                                     │
│  Agent: Researcher                                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e011ed7b-2623-4f1b-909f-1d1b36c71c6f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **SYNTHESIS** — Apple's stock has shown an upward trend over the past few days, recently         │
│  reaching a high of $258.79 before a slight decline. It maintains a significant market cap of approximately     │
│  $3.8 trillion, consistent with its leading position in the tech industry. Although the exact P/E ratio is not  │
│  available, the company's valuation relative to its historical performance can be check against industry norms  │
│  or analyst reports. Dividend payouts have been stable, showing moderate increments, reflecting a reliable      │
│  shareholder return strategy. No forthcoming corporate events on the horizon could act as immediate catalysts   │
│  for stock price movement, reducing near-term volatility risks.                                                 │
│                                                                                                                 │
│  **RECOMMENDATION** — HOLD: Given the stability in dividends, the resilience of its market cap, and the         │
│  absence of immediate market-moving events, it's prudent to adopt a hold strategy awaiting further fundamental  │
│  data to trigger a re-evaluation.                                                                               │
│                                                                                                                 │
│  **DATA SNAPSHOT** —                                                                                            │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "price_trend": "upward",                                                                                     │
│    "recent_price": 255.45,                                                                                      │
│    "market_cap": 3803334770730.35,                                                                              │
│    "dividends": [                                                                                               │
│      {"date": "2025-08-11", "dividend": 0.26},                                                                  │
│      {"date": "2025-05-12", "dividend": 0.26},                                                                  │
│      {"date": "2025-02-10", "dividend": 0.25},                                                                  │
│      {"date": "2024-11-08", "dividend": 0.25},                                                                  │
│      {"date": "2024-08-12", "dividend": 0.25}                                                                   │
│    ],                                                                                                           │
│    "upcoming_events": [],                                                                                       │
│    "recommendation": "HOLD"                                                                                     │
│  }                                                                                                              │
│  ```                                                  

In [26]:
# print each task result 
display(Markdown("# Planner Agent Output"))
display(Markdown(plan_research.output.raw))

# Planner Agent Output

**RESEARCH_PLAN** — numbered bullets  

1. **Price Context:**  
   - Use `yf_prices('AAPL', period='90d', interval='1d')` to gather daily price data for the last 90 days.  
   - Examine the trend for any noticeable fluctuations or patterns in the stock price that may inform your analysis.

2. **Key Fundamentals:**  
   - Access `yf_fundamentals('AAPL')` to review key company fundamentals including metrics such as PE ratio, market cap, and revenue.  
   - Identify any fundamental strengths or weaknesses compared to industry peers.

3. **Dividends:**  
   - Retrieve recent dividend data using `yf_dividends('AAPL', limit=5)`.  
   - Analyze these dividends to understand Apple's dividend yield, consistency, and any recent changes in payout.

4. **Upcoming Events:**  
   - Utilize `yf_calendar('AAPL')` to obtain information on upcoming events such as earnings announcements or shareholder meetings.  
   - Identify any events that may impact the stock price.

5. **Red Flag Checks:**  
   - Re-assess all gathered data for any red flags, such as sudden changes in fundamentals or dividends, or unusual price volatility.  
   - If necessary data is missing, consider extending the period or reviewing industry reports for additional insights.  

By following these steps, the Researcher will comprehensively gather necessary data on AAPL using Yahoo Finance tools, allowing them to build a well-informed investment thesis.

In [27]:
display(Markdown("# Researcher Agent Output"))
display(Markdown(research_ticker.output.raw))

# Researcher Agent Output

**SYNTHESIS** — Apple's stock has shown an upward trend over the past few days, recently reaching a high of $258.79 before a slight decline. It maintains a significant market cap of approximately $3.8 trillion, consistent with its leading position in the tech industry. Although the exact P/E ratio is not available, the company's valuation relative to its historical performance can be check against industry norms or analyst reports. Dividend payouts have been stable, showing moderate increments, reflecting a reliable shareholder return strategy. No forthcoming corporate events on the horizon could act as immediate catalysts for stock price movement, reducing near-term volatility risks.

**RECOMMENDATION** — HOLD: Given the stability in dividends, the resilience of its market cap, and the absence of immediate market-moving events, it's prudent to adopt a hold strategy awaiting further fundamental data to trigger a re-evaluation.

**DATA SNAPSHOT** —
```json
{
  "price_trend": "upward",
  "recent_price": 255.45,
  "market_cap": 3803334770730.35,
  "dividends": [
    {"date": "2025-08-11", "dividend": 0.26},
    {"date": "2025-05-12", "dividend": 0.26},
    {"date": "2025-02-10", "dividend": 0.25},
    {"date": "2024-11-08", "dividend": 0.25},
    {"date": "2024-08-12", "dividend": 0.25}
  ],
  "upcoming_events": [],
  "recommendation": "HOLD"
}
```